In [1]:
from jupyter_cadquery import show, open_viewer, set_defaults
import cadquery as cq
from build123d import *
cv = open_viewer("Build123d", cad_width=770, glass=True)
set_defaults(edge_accuracy=0.0001)

Overwriting auto display for cadquery Workplane and Shape


In [2]:
densa = 7800/1e6 #carbon steel density g/mm^3
densb = 2700/1e6 #aluminum alloy
densc = 1020/1e6 #ABS
ms = Mode.SUBTRACT
LMH, LMV = LengthMode.HORIZONTAL, LengthMode.VERTICAL

with BuildPart() as p:
    with BuildSketch(Plane.XZ) as s:
        with Locations((0,80)):
            Circle(60)
        with BuildLine(Plane.XZ) as l:
            m1 = Line((0,0),(115/2,0))
            m2 = Line(m1@1,m1@1+(0,10))
            m3 = PolarLine(m2@1,-115/2,180-20,length_mode=LMH)
            m4 = Line(m3@1,m1@0)
        make_face()
        split(bisect_by=Plane.XZ.offset(-80),keep=Keep.TOP)
        vtx = s.vertices().sort_by(Axis.X)[-4]
        fillet(vtx,5)
        mirror(about=Plane.YZ)
    extrude(amount=75/2,both=True)
    with BuildSketch(Plane.XZ) as s2:
        with Locations((0,80)):
            Circle(30/2)
    extrude(amount=75,both=True,mode=ms)
    with BuildSketch(Plane.XZ) as s3:
        with Locations((0,80)):
            Circle(55)
    extrude(amount=75/2-5,both=True,mode=ms)
    newp=Plane(origin=(115/2,0,10)).rotated((0,20,0))
    with BuildSketch(newp) as s4:
        with Locations((-6-5,0)):
            SlotOverall(45,20/2,90)
    zz=extrude(amount=-20,mode=Mode.PRIVATE)
    zz2 = mirror(zz,about=Plane.YZ,mode=ms)
    add(zz,mode=ms)

    edgs = p.part.edges().filter_by(Axis.Z).group_by(Axis.X)[-1] + p.part.edges().filter_by(Axis.Z).group_by(Axis.X)[0]
    fillet(edgs,20)

p